# Phân tích về đái tháo đường

## 1. Tổng quan về file csv
+ **Tổng quát:**
Hồ sơ bệnh nhân tiểu đường được thu thập từ hai nguồn, một thiết bị ghi điện tử tự động và hồ sơ giấy. 
    + Thiết bị tự động có đồng hồ bên trong để đánh dấu thời gian các sự kiện, trong khi hồ sơ giấy chỉ cung cấp các khoảng ‘thời gian logic’ (bữa sáng, bữa trưa, bữa tối, trước khi đi ngủ). 
    + Đối với hồ sơ giấy, thời gian cố định được gán cho các mốc: bữa sáng (08:00), bữa trưa (12:00), bữa tối (18:00) và trước khi đi ngủ (22:00). Do đó, hồ sơ giấy có các thời điểm ghi giả định đồng nhất, trong khi hồ sơ điện tử có dấu thời gian thực tế hơn

+ **Dữ liệu đầu vào:**
    + Number of times pregnant (số lần mang thai)
    + Plasma glucose concentration a 2 hours in an oral glucose tolerance test (Nồng độ glucose huyết tương sau 2 giờ trong nghiệm pháp dung nạp glucose bằng đường uống)
    + Diastolic blood pressure (mm Hg) (Huyết áp tâm trương)
    + Triceps skin fold thickness (mm) (Độ dày nếp gấp da cơ tam đầu hay cơ vùng mặt sau cánh tay)
    + 2-Hour serum insulin (mu U/ml) (huyết thanh insulin 2 giờ)
    + Body mass index (weight in kg/(height in m)^2) (chỉ số cơ thể)
    + Diabetes pedigree function (bệnh truyền từ gia đình)
    + Age (years) (tuổi)
+ **Kết quả:**
    + Class variable (0 or 1) (Lớp, phân biệt có bị bệnh hay không)

In [3]:
from PIL import Image
import matplotlib.pyplot as plt

## 2. Chuẩn bị vấn đề (Prepare Problem)

### 2.1 Khai báo thư viện

In [4]:
# Load libraries
import os, sys
from IPython import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import joblib

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

import warnings

%matplotlib inline
# plt.rcParams["figure.figsize"] = (12, 6)
# plt.rcParams['figure.dpi'] = 100

warnings.filterwarnings("ignore")

### 2.2. Nạp dữ liệu (Load Dataset)

In [ ]:
data_name = [    "pregnancies",        # Number of times pregnant
    "glucose",            # Plasma glucose concentration (2h OGTT)
    "blood_pressure",     # Diastolic blood pressure (mm Hg)
    "skin_thickness",     # Triceps skin fold thickness (mm)
    "insulin",            # 2-Hour serum insulin (mu U/ml)
    "bmi",                # Body mass index
    "diabetes_pedigree",  # Diabetes pedigree function
    "age",                # Age (years)
    "Class" ]
df_dataset = pd.read_csv(r"demo/lab02/pima-indians-diabetes.csv", names = data_names)